In [ ]:
import math
from os import path
from argparse import ArgumentParser
import random
from pathlib import Path
import pickle

import gpytorch
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.special import logsumexp

from alpaca.utils.ue_metrics import uq_ll
from alpaca.ue.masks import BasicBernoulliMask, DPPMask
import alpaca.nn as ann
from alpaca.utils.model_builder import uncertainty_mode, inference_mode
from alpaca.utils.datasets.builder import build_dataset


def manual_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)
    np.random.seed(seed)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
save_dir = Path('data/regression_4')

### params
lengthscale = 1e-2
T = 10000


def main(name, repeats, batch_size, sampler):
    manual_seed(42)
    dataset = build_dataset(name, val_split=0)
    x, y = dataset.dataset('train')
    N = x.shape[0] * 0.9  # train size

    #%%
    # best_tau, best_dropout = 0.01, 0.05
    best_tau, best_dropout = select_params(x, y, N, batch_size, name, sampler)

    #%%
    vanilla_rmse = []
    vanilla_ll = []
    mc_rmse = []
    mc_ll = []

    for i in range(repeats):
        manual_seed(42 + i)
        print(i)
        x_train, y_train, x_test, y_test, y_scaler = split_and_scale(x, y)

        model = build_and_train(
            x_train,
            y_train,
            500,
            best_tau,
            best_dropout,
            N,
            batch_size,
            split_num=i,
            name=name,
            sampler=sampler
        )

        error, ll = rmse_nll(model, 1, x_test, y_test, y_scaler, tau=best_tau, dropout=False)
        vanilla_rmse.append(error)
        vanilla_ll.append(ll)

        error, ll = rmse_nll(model, T, x_test, y_test, y_scaler, tau=best_tau, dropout=True)
        mc_rmse.append(error)
        mc_ll.append(ll)

    print('vanilla')
    print(np.mean(vanilla_rmse), np.std(vanilla_rmse))
    print(np.mean(vanilla_ll), np.std(vanilla_ll))
    print('mc')
    print(np.mean(mc_rmse), np.std(mc_rmse))
    print(np.mean(mc_ll), np.std(mc_ll))

    # plt.boxplot(vanilla_rmse)
    # plt.boxplot(mc_rmse)
    # plt.show()


class Network(nn.Module):
    def __init__(self, input_size, dropout_value, sampler):
        super().__init__()
        if sampler == 'mc':
            mask_class = DPPMask
        else:
            mask_class = BasicBernoulliMask

        base_size = 64

        self.dropout_1 = ann.Dropout(dropout_rate=dropout_value, dropout_mask=mask_class())
        self.fc1 = nn.Linear(input_size, base_size)

        self.dropout_2 = ann.Dropout(dropout_rate=dropout_value, dropout_mask=mask_class())
        self.fc2 = nn.Linear(base_size, 2*base_size)

        self.dropout_3 = ann.Dropout(dropout_rate=dropout_value, dropout_mask=mask_class())
        self.fc3 = nn.Linear(2*base_size, 2*base_size)

        self.dropout_4 = ann.Dropout(dropout_rate=dropout_value, dropout_mask=mask_class())
        self.fc4 = nn.Linear(2*base_size, 1)

    def forward(self, x):
        x = self.dropout_1(x)
        x = F.relu(self.fc1(x))

        x = self.dropout_2(x)
        x = F.relu(self.fc2(x))

        x = self.dropout_3(x)
        x = F.relu(self.fc3(x))

        x = self.dropout_4(x)
        x = self.fc4(x)
        return x


def loader(x_array, y_array, batch_size):
    dataset = TensorDataset(torch.Tensor(x_array), torch.Tensor(y_array))
    return DataLoader(dataset, batch_size=batch_size)


def rmse(values, predictions):
    return np.sqrt(np.mean(np.square(values - predictions)))


def rmse_nll(model, T, x_test, y_test, y_scaler, tau, dropout=True):
    y_test_unscaled = y_scaler.inverse_transform(y_test)
    model.eval()
    if dropout:
        uncertainty_mode(model)
    else:
        inference_mode(model)

    with torch.no_grad():
        y_hat = np.array([
            y_scaler.inverse_transform(
                model(torch.Tensor(x_test).to(device).double()).cpu().numpy()
            )
            for _ in range(T)
        ])

    y_pred = np.mean(y_hat, axis=0)
    errors = np.abs(y_pred - y_test_unscaled)
    ue = np.std(y_hat, axis=0) + 1/tau
    ll = uq_ll(errors, ue)
    # ll = np.mean((logsumexp(-0.5 * tau * (y_test_unscaled[None] - y_hat)**2., 0) - np.log(T)
    #         - 0.5*np.log(2*np.pi) + 0.5*np.log(tau)))
    return rmse(y_test_unscaled, y_pred), ll


def split_and_scale(x, y):
    # Load dat
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    # Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    y_scaler = StandardScaler()
    y_train = y_scaler.fit_transform(y_train)
    y_test = y_scaler.transform(y_test)

    return x_train, y_train, x_test, y_test, y_scaler


def build_and_train(x_train, y_train, epochs, tau, dropout_value, N, batch_size, split_num, name, sampler):
    if split_num is None:
        file_name = None
    else:
        file_name = save_dir / f"{name}_{sampler}_{split_num}.pt"

    reg = lengthscale**2 * (1 - dropout_value) / (2. * N * tau)
    train_loader = loader(x_train, y_train, batch_size)

    model = Network(x_train.shape[1], dropout_value, sampler).to(device).double()
    if file_name and path.exists(file_name):
        model.load_state_dict(torch.load(file_name))
        model.eval()
    else:
        model.train()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=reg)
        criterion = nn.MSELoss()
        train_losses = []

        for epoch in range(epochs):
            losses = []
            for x_batch, y_batch in train_loader:
                preds = model(x_batch.to(device).double())
                optimizer.zero_grad()
                loss = criterion(y_batch.to(device).double(), preds)
                loss.backward()
                optimizer.step()
                losses.append(loss.item())
            if epoch % 10 == 0:
                train_losses.append(np.mean(losses))
        if file_name:
            pass
            # torch.save(model.state_dict(), file_name)
    return model


def select_params(x, y, N, batch_size, name, sampler):
    file_name = save_dir/f"{name}_params_{sampler}.pickle"

    if path.exists(file_name):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        best_tau = params['tau']
        best_dropout = params['dropout']
    else:
        x_train, y_train, x_test, y_test, y_scaler = split_and_scale(x, y)
        results = []
        for local_tau in np.logspace(-4, 2, 14):
            for local_dropout in [0.05, 0.2, 0.5]:
                model = build_and_train(
                    x_train, y_train, 50, local_tau, local_dropout, N, batch_size, None, name, sampler
                )
                error, ll = rmse_nll(model, 1, x_test, y_test, y_scaler, dropout=False, tau=local_tau)
                results.append((ll, error, (local_tau, local_dropout)))
                print(results[-1])

        best_tau, best_dropout = sorted(results, key=lambda p: p[0])[-1][-1]
        print(best_tau, best_dropout)
        # with open(file_name, 'wb') as f:
        #     params = {'tau': best_tau, 'dropout': best_dropout}
            # pickle.dump(params, f)
    return best_tau, best_dropout



In [ ]:
# repeats = 1
name = 'boston_housing'

manual_seed(42)
dataset = build_dataset(name, val_split=0)
x, y = dataset.dataset('train')
N = x.shape[0] * 0.9  # train size


In [ ]:
# best_tau, best_dropout = 0.01, 0.05
# best_tau, best_dropout = select_params(x, y, N, batch_size, name, sampler)

# for i in range(repeats):
manual_seed(42)


In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_squared_error as mse
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def rmse_ll(true_y, prediction, y_scaler):
    return np.square(
        mse(
            y_scaler.inverse_transform(true_y),
            y_scaler.inverse_transform(prediction)
        )
    )


In [ ]:
x_train, y_train, x_test, y_test, y_scaler = split_and_scale(x, y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:

# for length_scale in np.logspace(-3, 1, 24):
# for restarts in [0, 3, 5, 10, 20, 50]:
restarts = 500
length_scale = 1e-2
kernel = C(1.0) * RBF(13*[length_scale])
model = GaussianProcessRegressor(alpha=1e-3, kernel=kernel, n_restarts_optimizer=restarts)
model.fit(x_train, y_train)
error = rmse_ll(y_test, model.predict(x_test), y_scaler)
print(error, restarts)


In [ ]:
preds, uq = model.predict(x_test, return_std=True)




In [ ]:
pred = np.mean(y_train) * np.ones(y_test.shape)
rmse_ll(y_test, pred, y_scaler)


class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [ ]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 50
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()


In [ ]:
test_x = torch.linspace(0, 1, 51).cuda()
model.eval().cuda()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    mean = observed_pred.mean
    lower, upper = observed_pred.confidence_region()